In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import scipy.signal as signal
from timeit import Timer
import timeit
import numba
from numba import jit
from numba import vectorize
import time

In [3]:
# All interesting comments
# 1. Input array type is np.ubyte however had to keep output type as np.int16 as 
#    np.int8 is not sufficient for the possible range. 

In [4]:
print (np.iinfo(np.ubyte))
print (np.iinfo(np.int16))

Machine parameters for uint8
---------------------------------------------------------------
min = 0
max = 255
---------------------------------------------------------------

Machine parameters for int16
---------------------------------------------------------------
min = -32768
max = 32767
---------------------------------------------------------------



In [5]:
kernel = np.array([-1, 0, 1])

In [6]:
def Dx_with_filter_specific_custom_impl(A):
    Dx = np.zeros_like(A, dtype=np.int16)
    Dx[:, 1:] -= A[:, :-1]
    Dx[:, :-1] += A[:, 1:]
    return Dx

# This performs very poor without numba!
def Dx_with_filter_specific_custom_impl_loops(A):
    Dx = np.zeros_like(A, dtype=np.int16)
    R, C = A.shape
    for i in range(R):
        for j in range(C):
            if j > 0:
                Dx[i,j] -= A[i, j-1]
            if j < C-1:
                Dx[i,j] += A[i,j+1]
    return Dx

def Dx_with_numpy_convolve(A):
    Dx = np.zeros_like(A, dtype=np.int16)
    for i in range(A.shape[0]):
        Dx[i] = np.convolve(A[i], kernel, mode='same')
    return Dx

@jit(nopython=True)
def numba_Dx_with_filter_specific_custom_impl(A):
    Dx = np.zeros_like(A, dtype=np.int16)
    R, C = A.shape
    # Changing the order of loops doesn't impact functionality but reduces the performance!!!
    for i in range(R):
        for j in range(C):
            if j > 0:
                Dx[i,j] -= A[i,j-1]
            if j < C-1:
                Dx[i,j] += A[i,j+1]
    return Dx

def Dx_with_scipy_convolve(A):
    Dx = signal.convolve2d(A, kernel.reshape(1, -1), mode='same')
    return Dx
    
# Done for validating my implementation :-), expect it not to be performant (at least in terms of memory usage)
def Dx_with_pytorch(A):
    ip = torch.from_numpy(np.expand_dims(A, axis=(0,1)).astype(np.int16))
    weight = torch.from_numpy(np.expand_dims(kernel, axis=(0,1,2)).astype(np.int16))
    return F.conv2d(ip, weight, padding=(0,1)).numpy()

In [7]:
def Dy_with_filter_specific_custom_impl(A):
    Dy = np.zeros_like(A, dtype=np.int16)
    Dy[1:, :] -= A[:-1, :]
    Dy[:-1, :] += A[1:, :]
    return Dy

@jit(nopython=True)
def numba_Dy_with_filter_specific_custom_impl(A):
    Dy = np.zeros_like(A, dtype=np.int16)
    R, C = A.shape
    for i in range(R):
        for j in range(C):
            if i > 0:
                Dy[i,j] -= A[i-1,j]
            if i < R-1:
                Dy[i,j] += A[i+1,j]
    return Dy

def Dy_with_numpy_convolve(A):
    Dy = np.zeros_like(A, dtype=np.int16)
    for i in range(A.shape[1]):
        Dy[:, i] = np.convolve(A[:, i], kernel.T, mode='same')
    return Dy

def Dy_with_scipy_convolve(A):
    Dy = signal.convolve2d(A, kernel.reshape(-1, 1), mode='same')
    return Dy
    
# Done for validating my implementation :-), expect it not to be performant (at least in terms of memory usage)
def Dy_with_pytorch(A):
    ip = torch.from_numpy(np.expand_dims(A, axis=(0,1)).astype(np.int16))
    weight = torch.from_numpy(np.expand_dims(kernel, axis=(0,1,3)).astype(np.int16))
    return F.conv2d(ip, weight, padding=(1,0)).numpy()

In [195]:
row, col = 7, 5
SA = np.random.randint(0, 255, size=(row, col), dtype=np.ubyte)
print (SA.dtype)
print (SA.shape)
print (SA)

uint8
(7, 5)
[[199 236   2   8 253]
 [122 203 231 156 113]
 [145 179 187 133 245]
 [ 28 200 201 113 165]
 [ 47 178 185  58  56]
 [ 10 202  88  82   8]
 [ 94  40  81 176  80]]


In [228]:
expected = Dx_with_pytorch(SA)
for func in [Dx_with_filter_specific_custom_impl, Dx_with_filter_specific_custom_impl_loops, numba_Dx_with_filter_specific_custom_impl]:
    if np.any(func(SA) - expected):
        print ('Output from function {} unexpected'.format(func.__name__))
print ('Done')

Done


In [227]:
expected = Dy_with_pytorch(SA)
for func in [Dy_with_filter_specific_custom_impl, numba_Dy_with_filter_specific_custom_impl]:
    if np.any(func(SA) - expected):
        print ('Output from function {} unexpected'.format(func.__name__))
print ('Done')

Done


In [122]:
row, col = 7000, 5000
LA = np.random.randint(0, 255, size=(row, col), dtype=np.ubyte)
print (LA.dtype)
print (LA.shape)

uint8
(7000, 5000)


In [217]:
# for func in [Dx_with_numpy_convolve, Dx_with_pytorch, Dx_with_scipy_convolve, 
#              Dx_with_filter_specific_custom_impl, numba_Dx_with_filter_specific_custom_impl]:
for func in [Dx_with_filter_specific_custom_impl, numba_Dx_with_filter_specific_custom_impl]:
    print ('Time from function {}'.format(func.__name__))
    t = Timer(lambda: func(LA))
    print (t.timeit(number=100))

Time from function Dx_with_filter_specific_custom_impl
6.201512004001415
Time from function numba_Dx_with_filter_specific_custom_impl
6.660915219999879


In [126]:
Dx = Dx_with_filter_specific_custom_impl(LA)

(7000, 5000)


In [240]:
print (Dx.shape)

(7000, 5000)


In [142]:
numba_Dx_with_filter_specific_custom_impl.signatures

[(array(uint8, 2d, C),)]

In [229]:
for func in [Dy_with_filter_specific_custom_impl, numba_Dy_with_filter_specific_custom_impl]:
    print ('Time from function {}'.format(func.__name__))
    t = Timer(lambda: func(LA))
    print (t.timeit(number=100))

Time from function Dy_with_filter_specific_custom_impl
7.240732628999467
Time from function numba_Dy_with_filter_specific_custom_impl
6.393693310999879


In [127]:
Dy = Dy_with_filter_specific_custom_impl(LA)
print (Dy.shape)

(7000, 5000)


In [8]:
def minmax_with_one_loop(A):
    A = A.ravel()
    n = A.size
    max_val = min_val = A[0]
    i = 1
    while i < n:
        e = A[i]
        if e < min_val:
            min_val = e
        elif e > max_val: # No need to check this condition when the element is already lowest
            max_val = e
    return min_val, max_val

@jit(nopython=True)
def numba_minmax_with_one_loop(A):
    min_val = max_val = A[0][0]
    R, C = A.shape
    for i in range(R):
        for j in range(C):
            e = A[i,j]
            if e < min_val:
                min_val = e
            elif e < max_val:
                max_val = e
    return min_val, max_val

@jit(nopython=True)
def numba_minmax_with_optimized_comparisons(A):
    A = A.ravel()
    n = A.size
    odd = n % 2
    if not odd:
        n -= 1
    max_val = min_val = A[0]
    i = 1
    while i < n:
        x = A[i]
        y = A[i + 1]
        if x > y:
            x, y = y, x
        min_val = min(x, min_val)
        max_val = max(y, max_val)
        i += 2
    if not odd:
        x = A[n]
        min_val = min(x, min_val)
        max_val = max(x, max_val)
    return min_val,max_val

def minmax_with_optimized_comparisons(A):
    A = A.ravel()
    n = A.size
    odd = n % 2
    if not odd:
        n -= 1
    max_val = min_val = A[0]
    i = 1
    while i < n:
        x = A[i]
        y = A[i + 1]
        if x > y:
            x, y = y, x
        min_val = min(x, min_val)
        max_val = max(y, max_val)
        i += 2
    if not odd:
        x = A[n]
        min_val = min(x, min_val)
        max_val = max(x, max_val)
    return min_val,max_val

def minmax_with_numpy_library(A):
    return np.min(A), np.max(A)

In [238]:
#for func in [minmax_with_numpy_library, numba_minmax_with_optimized_comparisons, minmax_with_optimized_comparisons, minmax_with_one_loop]:
for func in [minmax_with_numpy_library, numba_minmax_with_optimized_comparisons, numba_minmax_with_one_loop]:
    print ('Output from function {}'.format(func.__name__))
    print (func(Dx))

Output from function minmax_with_numpy_library
(-254, 254)
Output from function numba_minmax_with_optimized_comparisons
(-254, 254)
Output from function numba_minmax_with_one_loop
(-254, -254)


In [247]:
# for func in [minmax_with_numpy_library, numba_minmax_with_optimized_comparisons, minmax_with_optimized_comparisons, minmax_with_one_loop]:
for func in [minmax_with_numpy_library, numba_minmax_with_one_loop, numba_minmax_with_optimized_comparisons]:
    print ('Time from function {}'.format(func.__name__))
    t = Timer(stmt='func(IP)', setup='from __main__ import func; import numpy as np; IP = np.random.randint(low=-255, high=256, size=(1024,1024), dtype=np.int16)')
    n_calls, total_time = t.autorange()
    print ('Total time = {}, n_calls = {}, average = {}'.format(total_time, n_calls, total_time / n_calls))

Time from function minmax_with_numpy_library
Total time = 0.40338568999868585, n_calls = 500, average = 0.0008067713799973717
Time from function numba_minmax_with_one_loop
Total time = 0.20760240300296573, n_calls = 100, average = 0.0020760240300296572
Time from function numba_minmax_with_optimized_comparisons
Total time = 0.24735747200611513, n_calls = 2000, average = 0.00012367873600305758


In [250]:
%%timeit
IP = np.random.randint(low=-255, high=256, size=(1024,1024), dtype=np.int16)

4.11 ms ± 207 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [252]:
IP = np.random.randint(low=-255, high=256, size=(1024,1024), dtype=np.int16)

In [253]:
%%timeit
# IP = np.random.randint(low=-255, high=256, size=(1024,1024), dtype=np.int16)
_, _ = minmax_with_numpy_library(IP)

831 µs ± 84 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [254]:
%%timeit
# IP = np.random.randint(low=-255, high=256, size=(1024,1024), dtype=np.int16)
_, _ = numba_minmax_with_optimized_comparisons(IP)

126 µs ± 9.46 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [105]:
%%timeit
Dx_with_scipy_convolve(LA)

7.31 ms ± 466 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [106]:
%%timeit
Dx_with_filter_specific_custom_impl(LA)

488 µs ± 6.83 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [9]:
input_arrays_small = [np.random.randint(low=-255, high=256, size=(16,16), dtype=np.int16) for _ in range(1000)]

In [10]:
input_arrays_med = [np.random.randint(low=-255, high=256, size=(512,512), dtype=np.int16) for _ in range(100)]

In [17]:
input_arrays_large = [np.random.randint(low=-255, high=256, size=(4096,4096), dtype=np.int16) for _ in range(20)]

In [23]:
def create_small_arrays():
    return [np.random.randint(low=-255, high=256, size=(16,16), dtype=np.int16) for _ in range(1000)]
def create_med_arrays():
    return [np.random.randint(low=-255, high=256, size=(512,512), dtype=np.int16) for _ in range(100)]
def create_large_arrays():
    return [np.random.randint(low=-255, high=256, size=(4096,4096), dtype=np.int16) for _ in range(20)]

def measure(f, input_arrays_lambda, threshhold=0.4):
    total = 0.0
    iterations = 0
    while total < threshhold:
        input_arrays = input_arrays_lambda()
        start = time.perf_counter()
        for ip in input_arrays:
            f(ip)
        stop = time.perf_counter()
        total += stop - start
        iterations += 1
    loop = len(input_arrays)
    avg_time_per_iteration = total/iterations
    avg_time_per_loop = avg_time_per_iteration/loop
    print ('Total Time = {}, Iterations = {}, Average per iteration = {}, Loop = {}, Average per 1 call = {}'
           .format(total, iterations, avg_time_per_iteration, loop, avg_time_per_loop))

In [24]:
measure(numba_minmax_with_optimized_comparisons, create_small_arrays)
measure(minmax_with_numpy_library, create_small_arrays)

Total Time = 0.4001055129995166, Iterations = 614, Average per iteration = 0.0006516376433216883, Loop = 1000, Average per 1 call = 6.516376433216883e-07
Total Time = 0.4096119699997871, Iterations = 20, Average per iteration = 0.020480598499989355, Loop = 1000, Average per 1 call = 2.0480598499989355e-05


In [25]:
measure(numba_minmax_with_optimized_comparisons, create_med_arrays)
measure(minmax_with_numpy_library, create_med_arrays)

Total Time = 0.40576040100006594, Iterations = 71, Average per iteration = 0.005714935225353042, Loop = 100, Average per 1 call = 5.714935225353042e-05
Total Time = 0.4107095440000421, Iterations = 15, Average per iteration = 0.027380636266669475, Loop = 100, Average per 1 call = 0.00027380636266669477


In [26]:
threshhold = 10.0
measure(numba_minmax_with_optimized_comparisons, create_large_arrays, threshhold)
measure(minmax_with_numpy_library, create_large_arrays, threshhold)

Total Time = 10.083471972000325, Iterations = 70, Average per iteration = 0.14404959960000463, Loop = 20, Average per 1 call = 0.0072024799800002315
Total Time = 10.007752615999834, Iterations = 29, Average per iteration = 0.34509491779309776, Loop = 20, Average per 1 call = 0.017254745889654886
